In [1]:
using Pkg; Pkg.update

using ParallelTemperingMonteCarlo,BenchmarkTools,StaticArrays

Looking into the implementation of the EAM. A semi-two-body potential 

In [2]:
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
[0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
[0.1811648830,        0.5575692066,        0.2931316748],
[0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
[0.4742970144,        0.3445967202,       -0.2931309590],
[0.4742970205,       -0.3445967231,       -0.2931309559],
[0.0000007214,       -0.0000000073,       -0.6554619185]]
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]

nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13
pos_cu55 = copperconstant*ico_55
AtoBohr = 1.8897259886
bc_cu13 = SphericalBC(radius=8*AtoBohr)
start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-3.35737368666e-8, -3.01478453496e-8, -2.2610884012200003e-8], [-5.0100237544608e-6, -9.5924962476e-9, 4.491082807389623], [1.2413025402138254, -3.8203432492477134, 2.0084746703815606], [-3.2497803034902475, -2.3611020206489175, 2.00846968707976], [-3.249780345286124, 2.3611019212980637, 2.0084696658392325], [1.241302471695995, 3.8203432300627207, 2.0084746361226458], [4.016944341017041, 1.50739226748e-8, 2.008477729017507], [-1.2413025388434689, -3.8203432896732337, -2.0084746806592353], [-4.016944408164515, -7.468443507060001e-8, -2.008477773554097], [-1.2413026073612992, 3.8203431896372013, -2.0084747149181505], [3.2497802363427737, 2.361101961038405, -2.00846973161635], [3.2497802781386502, -2.361101980908576, -2.0084697103758224], [4.9428762807276005e-6, -5.0018016148200005e-8, -4.491082852611391]], SphericalBC{Float64}(228.54811596737582))

Generating the necessary dist2_matrix

In [3]:
dis2mat = [distance2(a,b) for a in pos_cu13, b in pos_cu13]
dist2mat_55 = [distance2(a,b) for a in pos_cu55, b in pos_cu55]

55×55 Matrix{Float64}:
  0.0      20.939    20.939    20.939   …   62.2806   62.2806   82.1728
 20.939     0.0      23.1496   23.1496     144.657   144.657   186.072
 20.939    23.1496    0.0      23.1496     121.19     83.2196  140.213
 20.939    23.1496   23.1496    0.0        144.657   121.19    140.213
 20.939    23.1496   60.6064   23.1496     121.19    144.657   140.213
 20.939    23.1496   60.6064   60.6064  …   83.2196  121.19    140.213
 20.939    23.1496   23.1496   60.6064      83.2196   83.2196  140.213
 20.939    60.6064   23.1496   23.1496      83.2196   45.249    66.0106
 20.939    60.6064   60.6064   23.1496      83.2196   83.2196   66.0106
 20.939    60.6064   83.756    60.6064      45.249    83.2196   66.0106
  ⋮                                     ⋱                      
 82.1728  140.213   140.213   186.072       22.7447   69.2332   90.848
 62.2806  121.19     83.2196  144.657       23.7891   23.7891   69.2332
 82.1728  140.213    66.0106  140.213       69.2332   22

The mathematical form of the EAM is:

$$
E = \frac{1}{2}\sum_{i,j :i \neq j}\epsilon\left(\frac{a}{r_{ij}}\right)^n - \epsilon C \sum_{i}\left[\sum_{j\neq i} \left(\frac{a}{r_{ij}}\right)^m \right]^{1/2}
$$

We have two terms, one similar to the LJ repulsion term, one a density term $F(\rho _i)$.

We can consider that $\frac{\epsilon a^n}{2}$ and $\epsilon C a^{m/2} $ can be taken out of the sum. Thus a single sum represents contracting a vector in the dist matrix. 

In [4]:
# struct EmbeddedAtomPotential <: AbstractDimerPotential
#     n::Float64
#     m::Float64
#     ean::Float64
#     eCam::Float64
# end
# function EmbeddedAtomPotential(n,m,ϵ,C,a)
#     epsan = ϵ * a^n / 2
#     epsCam = ϵ * C * a^(m/2)
#     return EmbeddedAtomPotential(n,m,epsan,epsCam)
# end

In [5]:
evtohartree = 0.0367493

n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561


copperpot = EmbeddedAtomPotential(n,m,ϵ,C,a)

suttonchenpot = EmbeddedAtomPotential(9.0,6.0,0.0126*evtohartree,39.432,0.3612*nmtobohr)

copperpot2 = EmbeddedAtomPotential(9.05 , 5.005 , 0.0225*evtohartree,33.17,0.327*nmtobohr)

EmbeddedAtomPotential(9.05, 5.005, 5949.446479587037, 2.6153101861477057)

In [6]:
invrexp(r2,expterm) = ifelse(r2 != 0 , 1/sqrt(r2)^expterm, 0. )
calcvec(distvec,expterm) = sum(invrexp.(distvec,Ref(expterm)))

function energyatomi(distvec,pot::EmbeddedAtomPotential) 
    Fterm = pot.eCam*sqrt(calcvec(distvec,pot.m))
    phiterm = pot.ean*calcvec(distvec,pot.n)
    return phiterm - Fterm
end
function energytotal(distmatrix,pot::EmbeddedAtomPotential)
    total_energy = 0.
    for distrow in eachrow(distmatrix)
        total_energy += energyatomi(distrow,pot)
    end
    return total_energy 
end
E_vec = [energyatomi(distrow,copperpot) for distrow in eachrow(dis2mat)]
sum(E_vec)

E_vec2 = [energyatomi(distrow,suttonchenpot) for distrow in eachrow(dis2mat)]
sum(E_vec2)

-1.3002351409450545

In [7]:
energytotal(dis2mat,suttonchenpot)

-1.3002351409450545

In [8]:
@benchmark energytotal($dist2mat_55,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  35.061 μs …  4.232 ms  ┊ GC (min … max): 0.00% … 98.14%
 Time  (median):     47.056 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.341 μs ± 89.483 μs  ┊ GC (mean ± σ):  4.18% ±  2.20%

   ▂▆█▆             ▁▃▆▄▃▁                                     
  ▃█████▄▃▂▂▂▁▁▁▁▂▃▆███████▆▅▄▃▃▂▂▂▂▂▂▁▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  35.1 μs         Histogram: frequency by time          72 μs <

 Memory estimate: 53.28 KiB, allocs estimate: 110.

# An alternative approach

Considering that the $\rho_i$ term depends on the square of sums, we can't simply update the total energy by considering the change to a single $r_{ij}$ on the global energy.

Rather than calculating $E_i$ and being unable to _separate_ the terms, we can store the $\phi_i,\rho_i$ terms. This may result in slower calculation of the global terms, but it may speed up the calculation of $E_{new}$ at each step.

In [9]:
function invrexp(r2,n,m)
    if r2 != 0.
        r_term = 1/sqrt(r2)
        return r_term^n , r_term^m
    else
        return 0. , 0.
    end    
end

function calc_components(eatomvec,distancevec,n,m)
    for dist in distancevec
        eatomvec .+= invrexp(dist,n,m)
    end
    return eatomvec
end
function calc_all_components(dist2_mat,n,m)
    n_atoms = size(dist2_mat)[1]
    component_vector = zeros(n_atoms,2)
    for row_index in 1:n_atoms
        component_vector[row_index,:] = calc_components(component_vector[row_index,:],dist2_mat[row_index,:],n,m)
    end
    return component_vector
end
function calc_energies_from_components(component_vector,ean,ecam)
    return sum(ean.*component_vector[:,1] .- ecam*sqrt.(component_vector[:,2]))
end
function calc_total_energy(dist2_mat,pot::EmbeddedAtomPotential)
    componentvec = calc_all_components(dist2_mat,pot.n,pot.m)
    return componentvec,calc_energies_from_components(componentvec,pot.ean,pot.eCam) 
end

calc_total_energy (generic function with 1 method)

In [10]:
phirho,energytot = calc_total_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

In [11]:
@benchmark calc_total_energy($dist2mat_55,$copperpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  116.409 μs …  5.431 ms  ┊ GC (min … max): 0.00% … 97.17%
 Time  (median):     126.580 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   129.117 μs ± 90.757 μs  ┊ GC (mean ± σ):  1.21% ±  1.68%

          █    ▅  ▂▄▁                                           
  ▄▂▁▁▇▂▁▂██▅▅██▇▆███▇▇▇▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  116 μs          Histogram: frequency by time          153 μs <

 Memory estimate: 34.34 KiB, allocs estimate: 116.

In [12]:
function update_components(componentvec,atomindex,old_r2_vec,new_r2_vec,n,m)
    new_component_vec= copy(componentvec)

    for j_index in eachindex(new_r2_vec)
        j_term = invrexp(new_r2_vec[j_index],n,m) .- invrexp(old_r2_vec[j_index],n,m)
        new_component_vec[j_index,:] .+= j_term 
        new_component_vec[atomindex,:] .+= j_term 
    end
    return new_component_vec
end

update_components (generic function with 1 method)

In [13]:
testindex,test_disp = 3,[0.5,0.5,-0.5]
test_pos = pos_cu55[testindex] + test_disp

3-element Vector{Float64}:
  1.764752985534391
 -3.3925157884546686
  1.5464230925690106

In [14]:
newdisvector = [distance2(test_pos,b) for b in pos_cu55]
newdisvector[testindex] = 0.

testenergyd2mat = copy(dist2mat_55)
testenergyd2mat[3,:] = newdisvector
testenergyd2mat[:,3] = newdisvector

newphirho,newentot = calc_total_energy(testenergyd2mat,suttonchenpot)

new_component_vector = update_components(phirho,3,dist2mat_55[3,:],newdisvector,9.,6.)
enew = calc_energies_from_components(new_component_vector,suttonchenpot.ean,suttonchenpot.eCam)

-6.0788077775169675

In [15]:
function calc_new_energy(componentvec,atomindex,old_r2,new_r2,pot::EmbeddedAtomPotential)
    new_component_vector = update_components(componentvec,atomindex,old_r2,new_r2,pot.n,pot.m)
    newenergy = calc_energies_from_components(new_component_vector,pot.ean,pot.eCam)

    return new_component_vector , newenergy 
end

calc_new_energy (generic function with 1 method)

In [16]:
calc_new_energy(phirho,3,dist2mat_55[3,:],newdisvector,suttonchenpot)[2]

-6.0788077775169675

In [17]:
@benchmark calc_new_energy($phirho,$3,$dist2mat_55[3,:],$newdisvector,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.002 μs … 699.688 μs  ┊ GC (min … max):  0.00% … 98.61%
 Time  (median):     3.609 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.047 μs ±  21.093 μs  ┊ GC (mean ± σ):  13.10% ±  3.12%

   ▁█▂                                                         
  ▂███▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▅▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3 μs            Histogram: frequency by time        8.46 μs <

 Memory estimate: 12.48 KiB, allocs estimate: 117.

# Finishing up

With the majority of the functions complete, it's just a matter of including them in the main code. Specifically how this is done is never the most fun challenge, but something to work on nevertheless.

In [18]:
get_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

# Incorporating this  into MCRun

With the energy functions complete, it's time to migrate this to mc_cycles. Because we have the additional output, we need to tailor the move function as well. I think the next goal is to create a new way to handle these data (change vectors within the states struct to prevent constant redefinition)

In [19]:
n_atoms = 55

ti = 550
tf = 900

n_traj = 20

temp = TempGrid{n_traj}(ti,tf)
mc_cycles = 1000
mc_sample = 1
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
move_strat = MoveStrategy(atom_moves = n_atoms)  
ensemble = NVT(n_atoms)
bc_cu55 = SphericalBC(radius=14*AtoBohr)
start_config = Config(pos_cu55, bc_cu55)
n_bin = 100
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, suttonchenpot) for i in 1:n_traj]
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Matrix{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [20]:
ptmc_run!((mc_states, move_strat, mc_params, suttonchenpot, ensemble, results); save=false)
results.en_max

equilibration done


MC loop done.
cc


[0.10641029213903545, 0.17245945019129752, 0.2403472398550044, 0.20767549006796748, 0.2749973179916749, 0.3061988093205824, 0.3786797142685972, 0.3531910871097771, 0.3496989163142619, 0.5225027498833525, 0.9628014097310654, 0.3211985282622963, 0.5992893398566448, 0.47711483597830484, 0.5610160195587709, 0.47720842528584845, 0.5991399860132391, 0.9230107194888424, 0.6560267440743665, 0.4287640982596773]
done


-5.926892704262755

In [27]:
pos_cu78 = [[9.20810577, 10.6811792, 12.85775222],
[7.72518083, 8.64747448, 13.47423093],
[8.83017234, 6.48185841, 14.26401845],
[5.76926904, 8.5204058, 11.78510919],
[10.3058934, 8.49163418, 13.72488251],
[11.0445983, 6.10374679, 13.03249449],
[10.76908039, 11.68540658, 11.04398002],
[6.86491181, 6.33673147, 12.63535963],
[11.14520009, 4.41652378, 11.06840909],
[10.56131156, 9.20976286, 11.36608831],
[3.8350787, 9.66043706, 10.62564335],
[8.18461754, 11.86457693, 10.78680942],
[12.53103163, 8.13651712, 12.41914688],
[9.10050302, 7.21655723, 11.96507541],
[8.9196861, 4.77149686, 12.37507079],
[8.03726493, 9.35893297, 11.12552957],
[11.75404842, 10.48292508, 13.05807496],
[7.19140363, 7.0877584, 10.30045396],
[6.63954366, 10.80307617, 12.52504858],
[13.20475821, 5.78540789, 11.69596475],
[10.04929714, 10.21021383, 2.74162567],
[8.60808677, 8.21325894, 3.29147489],
[11.13551339, 8.08991071, 3.51785384],
[7.18684228, 6.21910921, 3.93796442],
[7.88437782, 10.5718392, 3.95937956],
[9.71784555, 6.07918378, 4.09394596],
[11.97420313, 10.35137104, 4.33933113],
[6.42017871, 8.55796232, 4.53249281],
[9.97169751, 11.88885922, 4.62524758],
[9.82411407, 9.43691954, 5.10406749],
[5.02772356, 6.55318086, 5.20359556],
[5.71367449, 10.90921526, 5.25590498],
[8.41435724, 7.42843168, 5.68181186],
[11.0285349, 7.27653846, 5.85794355],
[7.79544252, 12.26033475, 5.87610665],
[4.27718147, 8.88880172, 5.86736713],
[11.91867483, 12.00967502, 6.24563146],
[6.9561607, 5.37238684, 6.26938662],
[7.67700429, 9.79132879, 6.32467407],
[9.50228326, 5.2069596, 6.45258385],
[11.87164769, 9.54494266, 6.68244807],
[12.02789873, 5.12581568, 6.74109416],
[9.88844903, 13.49922651, 6.57922127],
[3.59896936, 11.21261099, 6.58760866],
[6.25574475, 7.76131662, 6.94310978],
[9.77298468, 11.08697928, 7.00896014],
[5.64304702, 12.56406942, 7.20157052],
[9.65009283, 8.66311539, 7.41738673],
[12.95719784, 7.37463975, 7.57489662],
[3.97662306, 6.96385649, 7.4780558],
[5.5593229, 10.10255239, 7.6386882],
[7.73820851, 13.84622135, 7.8527848],
[8.1830049, 6.63840522, 8.0168064],
[10.70190431, 6.53159817, 8.22675048],
[13.72586678, 9.68714235, 8.3981167],
[7.98154103, 4.19765578, 8.32731731],
[3.27977734, 9.3022518, 8.18896381],
[7.62359206, 11.43808642, 8.27752581],
[10.55129337, 4.0805155, 8.5609616],
[11.76414234, 11.28499926, 8.65854603],
[5.91012759, 5.75557787, 8.56871289],
[7.49034308, 9.0057639, 8.68353281],
[11.54936709, 8.80691453, 9.05320905],
[9.7137393, 12.78636073, 8.95977573],
[12.67456914, 5.40926942, 9.22361494],
[4.12936341, 11.58348871, 9.01710543],
[5.22409251, 8.16647139, 9.25872611],
[9.56094879, 10.34182511, 9.36067818],
[9.37880896, 7.93821131, 9.68957149],
[6.19508081, 12.92482629, 9.64387612],
[13.5372612, 7.72382715, 10.06363071],
[4.95459032, 6.27620264, 10.88163222],
[9.20140742, 5.5571462, 10.04127471],
[6.0873292, 10.48531258, 10.10256552],
[6.96384327, 4.65312172, 10.66958722],
[9.02696245, 3.15942277, 10.40158812],
[11.28333689, 6.86978501, 10.68020117],
[12.79744324, 10.11163506, 10.75172547]]
pos_cu78.*=nmtobohr./10
cofm = [sum([pos[1] for pos in pos_cu78]),sum([pos[2] for pos in pos_cu78]),sum([pos[3] for pos in pos_cu78])]./78

for element in pos_cu78
    element .-=cofm 
end
println(pos_cu78)

[[0.47560863923076724, 2.097601591666667, 4.284427189615382], [-1.007316300769232, 0.06389687166666569, 4.900905899615381], [0.09767520923076844, -2.101719198333334, 5.690693419615382], [-2.963228090769232, -0.06317180833333325, 3.211784159615382], [1.5733962692307681, -0.0919434283333338, 5.1515574796153825], [2.3121011692307682, -2.479830818333334, 4.4591694596153815], [2.036583259230767, 3.1018289716666665, 2.4706549896153813], [-1.8675853207692326, -2.246846138333334, 4.062034599615382], [2.412702959230767, -4.167053828333334, 2.4950840596153814], [1.8288144292307678, 0.6261852516666657, 2.7927632796153823], [-4.897418430769232, 1.0768594516666656, 2.0523183196153827], [-0.5478795907692326, 3.280999321666666, 2.213484389615383], [3.798534499230767, -0.44706048833333334, 3.8458218496153815], [0.3680058892307674, -1.3670203783333337, 3.3917503796153827], [0.18718896923076755, -3.812080748333334, 3.8017457596153825], [-0.6952322007692331, 0.7753553616666657, 2.5522045396153814], [3.02

In [28]:
dismat78 = [distance2(a,b) for a in pos_cu78, b in pos_cu78]
println(get_energy(dismat78,suttonchenpot)[2])
energytotal(dismat78,suttonchenpot)

1159.4240958269834


1159.4240958269834

In [47]:
dismat78 = [distance2(a,b) for a in pos_cu78, b in pos_cu78]
println(get_energy(dismat78,suttonchenpot)[2])
energytotal(dismat78,copperpot)

-8.782894365702242


-8.844963437603418

In [46]:
testmat = [0.0010870780        0.0008983966       -0.0000000091
-0.0006767403        0.0009347027        0.7108737478
 0.1866074577       -0.5688467650        0.3590297930
-0.4831745856       -0.3508236247        0.3593186042
-0.4820665019        0.3540210238        0.3593385373
 0.1885276690        0.5702340772        0.3587961459
 0.5965541665        0.0003318705        0.3637214804
 0.5965541619        0.0003318696       -0.3637215062
 0.1885276644        0.5702340764       -0.3587961679
-0.4820665064        0.3540210229       -0.3593385502
-0.4831745901       -0.3508236256       -0.3593186154
 0.1866074531       -0.5688467658       -0.3590298122
-0.0006767494        0.0009347010       -0.7108737660
-0.0010022965        0.0004683799        1.3894611097
 0.1856258632       -0.5668289510        1.0554200156
-0.4824120982       -0.3500370683        1.0564363223
-0.4816194167        0.3528628504        1.0565969111
 0.1864201330        0.5676196653        1.0560583601
 0.5953469039       -0.0008611988        1.0594222175
 0.7885276948       -0.5721086450        0.7145880088
 0.3689793385       -1.1291887054        0.6888642926
-0.2988963422       -0.9226493037        0.7159100003
-0.9601068266       -0.6965765491        0.6891680452
-0.9692566632        0.0022580217        0.7158985678
-0.9579933003        0.7011150614        0.6891874276
-0.2961287134        0.9251019721        0.7159532233
 0.3724143387        1.1296213667        0.6890713543
 0.7882164696        0.5703368844        0.7168154465
 1.1913025021       -0.0042451730        0.7036315359
 0.7749817434       -0.5623953497       -0.0000000134
 0.3744906585       -1.1452102539       -0.0000000102
-0.2933666363       -0.9061455550       -0.0000000062
-0.9733355546       -0.7062112433       -0.0000000021
-0.9516959247        0.0023085807       -0.0000000030
-0.9711404658        0.7108585707       -0.0000000037
-0.2905063702        0.9087677956       -0.0000000083
 0.3784956079        1.1462271064       -0.0000000128
 0.7779260323        0.5634061314       -0.0000000147
 1.1839865614        0.0018426795       -0.0000000166
 1.1913024932       -0.0042451746       -0.7036315693
 0.7882164605        0.5703368828       -0.7168154761
 0.3724143300        1.1296213651       -0.6890713799
-0.2961287226        0.9251019704       -0.7159532398
-0.9579933091        0.7011150598       -0.6891874353
-0.9692566724        0.0022580200       -0.7158985736
-0.9601068354       -0.6965765507       -0.6891680496
-0.2988963513       -0.9226493054       -0.7159100126
 0.3689793297       -1.1291887070       -0.6888643128
 0.7885276857       -0.5721086466       -0.7145880358
 0.5953468904       -0.0008612013       -1.0594222433
 0.1864201196        0.5676196629       -1.0560583820
-0.4816194302        0.3528628480       -1.0565969240
-0.4824121117       -0.3500370707       -1.0564363336
 0.1856258497       -0.5668289535       -1.0554200349
-0.0010023142        0.0004683767       -1.3894611279
 1.3711742722       -0.5691061896        0.3418584627
 1.3733309984        0.5645337125        0.3534581081
 1.3711742678       -0.5691061904       -0.3418584970
 1.3733309939        0.5645337117       -0.3534581452
 0.9619770880       -1.1159749807        0.3416538517
 0.9656941684        1.1149445949        0.3423355120
 0.9619770836       -1.1159749815       -0.3416538795
 0.9656941640        1.1149445941       -0.3423355451
-0.1188949464       -1.4646232915        0.3417066024
-0.1142681695        1.4665373479        0.3417529660
-0.1188949508       -1.4646232923       -0.3417066157
-0.1142681739        1.4665373471       -0.3417529862
-0.7592730575        1.2589346266        0.3417447723
-0.7631850372       -1.2549718175        0.3417282363
-0.7631850415       -1.2549718183       -0.3417282418
-0.7592730619        1.2589346258       -0.3417447838
-1.4294471927       -0.3357258352        0.3417024565
-1.4284005055        0.3417488995        0.3417088584
-1.4294471971       -0.3357258360       -0.3417024557
-1.4284005098        0.3417488987       -0.3417088592
 1.7659227091       -0.0159569223        0.3392069583
 1.7659227048       -0.0159569231       -0.3392069989
 1.9300865282        0.5240552880       -0.0000000220
]*copperconstant
testingpos = []
for row in eachrow(testmat)
    push!(testingpos,Vector(row))
end
testdismat = [distance2(a,b) for a in testingpos, b in testingpos]

energytotal(testdismat,copperpot)

-8.947717041062216

Somehow we are getting clusters with significantly lower energy than the supposed minimum. Why that's the case is a little arbitrary and unclear. 

In [64]:
# testingmat2= [5.221544896276363 7.26605905771793 0.3378472540298592 
# 2.834277168518123 5.0128987601911374 0.24181777257730397 
# -3.321686655460808 4.812502665040112 0.7654312114505999 
# -5.566608261884243 -3.661638560457533 0.48262737570369374 
# 2.563988095168612 3.799401466678454 -2.875992535232376 
# -5.036839054063972 -3.760356202209659 3.7389202054380917 
# -1.0230042128303258 3.4505988567808874 -7.462567434031769 
# -4.68788988934847 0.6214218744962235 -1.64556128372929 
# 5.636078919027499 3.147936415697662 5.753799362101342 
# -6.711948641135058 -2.1963118360857363 -2.6720417545309254 
# -5.340378176815574 2.0405697710536077 1.3351047330431278 
# -3.2578942961468305 -2.836626966661775 -2.4161733164714745 
# 2.513127229157005 0.4482331020206985 7.829232414829937 
# -1.2734258787887502 -0.36756771156872514 6.838866434307665 
# -3.9820428537962282 -0.5926930473113958 4.958056475773787 
# -1.340636648071651 8.248573347214265 4.00618163663839 
# 2.5571088868199454 7.947311477298748 -1.7618552955953897 
# 4.644931661011133 -0.0723887954092481 -2.8241267420549994 
# -0.5577913105867545 -0.14116324039714792 -6.660943830410107 
# -1.6697900527346448 -0.1556868009223808 -3.5104236231607953 
# 3.083934376074827 0.889189212617502 4.548387975005215 
# -4.1243214764452265 8.379792436874219 1.3416508896465462 
# 7.599361086646662 1.1830207030440578 -0.5212080389272573 
# -0.10084633267386628 2.214614214912965 9.154944148370943 
# 1.0826811958057716 -3.747512650766421 2.2414720090675773 
# 0.03737857899761657 2.3785003668124225 5.221711276767121 
# -6.438974478713019 5.497159643658539 2.1619079496648386 
# -2.663434022180854 5.4143758050837265 -5.130777883313608 
# 4.660024820121328 5.907717079853802 -4.366536821112291 
# 0.36267945755550174 6.373073741109338 -3.976984114835602 
# 6.178166785545661 1.0491947498338894 3.022945570753405 
# 2.059560249564408 1.0872154055758205 -5.268471743433928 
# -3.6098578024058394 3.5404821274406713 -2.585689070616134 
# -5.791241736950104 5.9789249992604185 -1.2803319748343198 
# 4.630921941476251 -2.644799701254251 3.6765127605915287 
# 6.496397648094798 4.368405648785876 1.9397435713651843 
# 5.709917897097561 -1.5871142780142558 0.5802969948561519 
# -7.280080321719404 2.8921511147996606 -1.732480911956761 
# -1.4426069131684434 5.3111818341366055 7.358288609112606 
# 5.107899904620253 6.482689186413518 4.613157327680238 
# 2.194363455522978 4.162641560724317 -6.291419305444588 
# 1.4989938152262472 -2.0586591888191417 -3.912190997374886 
# -0.6899035940673387 -4.985713037050739 -1.8040236745897222 
# -0.33398318872487276 4.7176400197041355 -0.7681647486930668 
# -0.32329225395490424 -0.6312768600050563 3.2314918916865776 
# 2.213116069655553 7.6623243061084585 2.5518355738274643 
# -6.190750481020687 4.4535474361786465 -4.333711366276708 
# 0.8817039053181305 2.200953357895456 1.7169763787294623 
# -1.986783540083528 1.7134298765739644 -0.1256118619176756 
# -6.571997707073414 0.803547823194915 -4.350868522289296 
# 5.570272109995544 -0.4358013558902907 6.30244911177496 
# -2.159720680632099 -4.437828329453303 1.2886631994243258 
# 4.268443858273128 1.93411997086919 0.24033318399365114 
# 6.250630565478255 4.360402134472557 -1.5860912940941565 
# -2.7703446363540003 6.983165606689523 -1.8653906919018748 
# 2.5822530059043065 3.7588104277978225 7.027792826895864 
# -6.751668432997495 -0.6735905599313827 2.752849211374089 
# 2.5783628366281115 -0.8002942713044585 1.5901088597826283 
# 1.7724046053518476 -2.396322896204755 5.5315060563747185 
# -0.7023000119947479 8.051320170680867 0.7481649086573525 
# -1.306291887660633 -3.8678841640799906 -5.186463830357754 
# 2.4473046715519247 -3.8426914568941473 -0.7943330433930925 
# -0.366424255369754 5.308005864974367 2.6871454675979396 
# -3.5282800051771197 5.468041021558926 4.20009393861259 
# -3.74482707876014 1.9275638930115473 -5.723021134495188 
# -4.418275776299258 -1.6121482327530126 -5.407354532087365 
# -0.39949260442965373 -1.819006188611964 -0.30072681220122083 
# -7.8607828247821105 -0.25841811878290866 -0.46331553152827587 
# -1.5348253373824328 -3.685693857167294 4.561030129247325 
# 1.2969763082110934 1.088168072749807 -1.56265882081614 
# 3.0778180741755796 4.139244177126391 3.6571126923629196 
# -3.1159337485730454 2.577155211940834 6.676071866836851 
# -0.44975876845039797 2.802066623895416 -3.9825932493205656 
# 1.493669059930494 6.799820191859554 5.72063825557415 
# -3.3734036459705523 -1.114723385641742 1.3442116880459936 
# -5.980635262260103 2.3193374486937004 4.684951940798341 
# -2.5208111777520474 2.2051027384246495 3.0927016602529744 
# 5.614443388482977 2.8443871630935207 -4.884690172807231 ]
testingposition = [[-4.68609269479413, 16.6536106373705, -12.362317890632799], [10.273318189502785, -7.74635317851858, 15.642532990656406], [-10.83358069903258, -16.70473039388474, 15.433028182681104], [-4.918280403390947, 23.94388436973165, 0.6479319857423986], [1.6854617700735792, -2.19766938291046, -15.586422941677723], [-14.293116443268483, 13.063036602725841, -4.915040083219719], [-21.388932416063092, 6.821385126823539, -5.1946947782346875], [5.1585512310800015, 9.86385016499844, 21.75393482227623], [-14.710036832312662, 5.143779268958427, 8.3824277620554], [15.127895833935343, -2.9027428750944355, 15.516629454504255], [9.838409144138005, -4.58323378854489, 21.74148338074733], [-1.4794890931779792, 15.554373839476488, 11.51136580326892], [-20.233864832929207, -2.9608453916072444, 11.389230992903833], [-12.299228719871955, 7.4565991722791765, -14.510799228713207], [-17.45693238639266, 9.101747493343957, 1.747221900092384], [-19.16909187846023, -11.624077127291834, -13.783383339581999], [-2.7486510438873224, -20.73587277928711, 4.696585973861108], [-10.557941313687204, 13.002484659029404, 5.2568097954191995], [16.577959181731465, 9.413329277147247, -8.392205034158934], [-4.494689223457988, 9.105714236540896, 15.130376045719425], [-6.388499755109639, 4.811132968173485, 17.23091937235922], [-12.830545191880038, -3.8238142890915254, 3.7777316436505073], [5.402225750261561, 17.419199454637134, 1.4044164683148992], [-16.942250460343462, -9.39957746142342, 14.747119418293527], [2.6724596706154937, -1.3000294202938285, 20.336926942488063], [-10.35421108824254, -8.95404241694953, -4.826190471739455], [-22.228163556282563, 9.18362638222594, 5.558441795695744], [-20.09191394543307, -3.2831417491775072, -4.283685297335046], [-12.000920266363089, -11.373283787543595, 16.30828139616957], [13.081926243558033, 15.037351371567738, -8.05861572156113], [-10.563781364861292, -15.43809872919319, -13.762495574439345], [-6.17009175432412, -2.9319349922544937, -5.920546287020732], [12.374586077131536, -0.7216053472559962, 1.642603121513602], [21.56324719532919, 0.32560395737525605, -11.186742649572874], [11.933592249222214, 8.99059741972048, -10.769535404643031], [12.816237416760266, -3.687167179470609, -3.0514080943536213], [7.960387169001651, 0.13301847364713826, 17.02531296559485], [-3.32373913035435, -9.835715294202174, 10.548143521760261], [-4.424706060248219, 0.9759142032458525, -13.102589603610815], [-7.728833557010534, 5.0605811962995, -9.099586744744109], [1.8405798380685745, 15.907243975675263, -8.315680328687254], [13.170505053551002, -0.6726387421905038, -10.39472565172294], [15.526007332821871, 12.45995981637225, 1.1846780144177984], [7.714104621220535, 3.17452366908806, 1.1671159583543416], [8.3521881615766, -12.391082533287785, -9.799371750268774], [-15.85336113346709, -9.93997331251678, -5.849227138626227], [2.0950154589177776, -12.340886488463653, -18.304337913202993], [-9.122652279595705, -8.039543138348481, -22.79550542799303], [-2.8127210243747136, 23.951662281937146, -4.799589786974037], [-2.054094826980007, 10.558490258928032, 21.78087566717292], [-16.20307257333643, 2.4909125182187766, -19.929648532096895], [-7.572147101033449, 10.920101891313735, 10.473442519359093], [-16.521417990235147, -1.009338816133896, -15.82040998738849], [-19.66253614881819, -8.835120750849985, 9.567807264723513], [-12.027631957632599, -7.558009091286821, 20.11966187247257], [-4.688074841021177, -12.97237566786344, -1.8076896027333433], [0.6714047637662093, -21.528702720140103, -11.18389629588328], [-7.854374519873133, 15.535624335773925, -18.310445663204614], [5.685369884464817, -20.516462397512818, 1.3436815766278158], [17.97102027454319, -2.366133620553013, -15.695506880922853], [2.6108071911529107, 11.678575984512083, -11.408928669905537], [4.460839145376168, -15.36476590078325, -2.0345987356387543], [-2.622329445155356, -6.438671123258724, -10.361331297567775], [18.488285466996064, 10.069346393987034, 9.53763240337782], [1.3948966284783544, -12.789175293819053, -9.532928574568984], [-10.564900562878158, -1.5126731611752735, -14.364775181104989], [-0.32769314092093693, -5.558232939806113, -1.5601043380358686], [-7.427749924370372, -21.712858743697165, 1.3324789834346635], [5.774153484254598, 18.155422819517902, 14.213544652187757], [-10.866625671833887, -20.610603162910056, -4.6710014381678056], [10.781400077032288, 13.253606058766087, -15.202200681477224], [-1.4860283943908943, 8.90989287280117, 7.147017905994694], [16.600553159406388, -8.029135906397672, -10.114329526667825], [3.2932295612139093, 11.831800280889176, -17.653551918992292], [-10.331848024584486, 12.858397132453389, -7.754597862889901], [-3.326938701318986, 4.685663735764332, -19.82094962282251], [-13.915338119231821, 0.37969487087469433, -1.5647064372630952], [9.31456791686407, -21.524937632499977, 11.44756282135591]]
# testingposition=[]

# for row in eachrow(testingmat2)
#     push!(testingposition,Vector(row))
# end
distancesmatrix = [distance2(a,b) for a in testingposition,b in testingposition]
energytotal(distancesmatrix,copperpot)

-2.0904275572425552

In [65]:
energytotal(distancesmatrix,suttonchenpot)

-2.217680012663217

# Some issues

some issues have appeared. Namely, while the __states__ are correct, the __distance matrices__ are correct, the *components* are not, leading to incorrect energies.